In [69]:
from collections import defaultdict
from IPython.display import display
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk.stem.wordnet as wordnet
import numpy as np
import pandas as pd
import random
import sklearn
import string
import time

Requires grants_gov.csv and research_grant_history.csv

In [3]:
sw = set(stopwords.words('english'))
wnl = wordnet.WordNetLemmatizer()

In [4]:
pd.set_option('max_columns', 40)
grants_df = pd.read_csv('../scraped_data/grants_gov.csv', delimiter="~")
grants_df.head()

,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
0,262148,Establishment of the Edmund S. Muskie Graduate...,SCAPPD-14-AW-161-SCA-08152014,D,CA,O,Public Diplomacy,19.040,25,Eligibility for U.S. institutions is limited t...,DOS-SA,Bureau of South and Central Asian Affairs,8152014,9042014.0,8202014,600000.0,400000.0,600000.0,1.0,The Office of Press and Public Diplomacy of th...,Synopsis 2,No,10042014.0,DelageEM@state.gov,Inquiries,"Ellen Delage, Bureau of South and Central Asia...",NaN,NaN,NaN,NaN
1,262149,Eradication of Yellow Crazy Ants on Johnston A...,F14AS00402,D,CA,"AG,ENV,NR",NaN,15.608,99,The recipient has already been selected for th...,DOI-FWS,Fish and Wildlife Service,8152014,8222014.0,8152014,0.0,0.0,0.0,NaN,Funds under this award are to be used for the ...,Synopsis 1,No,8232014.0,Connie_Sauer@fws.gov,Connie_Sauer@fws.gov,"Connie Sauer, 503-490-0827&lt;br/&gt;Connie_Sa...",http://www.grants.gov/,http://www.grants.gov/,NaN,NaN
2,131073,"Cooperative Ecosystem Studies Unit, Piedmont S...",G12AS20003,D,CA,ST,NaN,15.808,25,This financial assistance opportunity is being...,DOI-USGS1,Geological Survey,11172011,11292011.0,11282011,0.0,0.0,31900.0,1.0,The USGS Southeast Ecological Science Center s...,Synopsis 2,No,12172011.0,fgraves@usgs.gov,fgraves@usgs.gov,"Faith Graves, 703-648-7356&lt;br/&gt;fgraves@u...",http://www.grants.gov/,http://www.grants.gov/,NaN,NaN
3,196613,OVW FY 2012 Training and Technical Assistance ...,OVW-2012-3380,D,CA,LJL,NaN,16.526,"06,12",NaN,USDOJ-OJP-OVW,Office on Violence Against Women,8292012,10152012.0,8292012,2500000.0,NaN,2500000.0,1.0,There are different statutory and regulatory r...,Synopsis 1,No,11142012.0,Christina.Murray@usdoj.gov,Christina.Murray@usdoj.gov,Christina Murray &lt;br/&gt;Phone 202-307-0344,http://www.ovw.usdoj.gov/docs/grantee-reportin...,OVW FY 2012 Training and Technical Assistance ...,CategoryExplanation,NaN
4,65553,Affordable Care Act (ACA): Childhood Obesity R...,RFA-DP-11-007,D,CA,HL,NaN,93.535,99,NaN,HHS-CDC-HHSCDCERA,Centers for Disease Control and Prevention - ERA,1192011,4252011.0,4152011,5250000.0,0.0,NaN,4.0,This Funding Opportunity Announcement (FOA) so...,Synopsis 10,No,5182011.0,pgotim@cdc.gov,General Email,Centers for Disease Control and Prevention (CD...,Please click on Full Announcement at the top o...,Please click on Full Announcement at the top o...,NaN,On-time submission requires that electronic ap...


In [5]:
faculty_df = pd.read_csv('../scraped_data/faculty_profiles.csv', delimiter="~")
faculty_df.head()

,faculty_name,faculty_profile_url,l_expertise,department,title_name,faculty_site_url,lab_url,faculty_email,description,description_links,article_date_1,title_of_news_1,link_to_news_1,description_teaser_1,article_date_2,title_of_news_2,link_to_news_2,description_teaser_2,article_date_3,title_of_news_3,link_to_news_3,description_teaser_3,article_date_4,title_of_news_4,link_to_news_4,description_teaser_4,article_date_5,title_of_news_5,link_to_news_5,description_teaser_5
0,David A. Aaker,/faculty/david-aaker,"business,marketing,branding",Haas School of Business,Professor of Marketing and Public Policy,http://www.haas.berkeley.edu/faculty/aaker.html,http://groups.haas.berkeley.edu/marketing/,aaker@haas.berkeley.edu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,"February 22, 2016",“Deep Learning”: A Giant Step for Robots,/bakarfellows/profile/pieter_abbeel,Bakar Fellow Pieter Abbeel studies deep lear...,"February 19, 2016",Three young faculty members honored by White ...,/news/three-young-faculty-members-honored-whit...,Three UC Berkeley faculty members named as r...,"May 21, 2015",New ‘deep learning’ technique enables robot m...,/news/new-deep-learning-technique-enables-robo...,UC Berkeley researchers have developed algor...,"December 17, 2012",Big NSF grant funds research into training ro...,/news/big-nsf-grant-funds-research-training-ro...,"What if robots and humans, working together,...","August 23, 2011",UC Berkeley robotics expert named among world...,/news/uc-berkeley-robotics-expert-named-among-...,"Pieter Abbeel, a UC Berkeley, professor know..."
2,Elizabeth Abel,/faculty/elizabeth-abel,"feminist theory,psychoanalysis,Virginia Woolf,...",Department of English,Professor of English,http://english.berkeley.edu/profiles/5,NaN,eabel@uclink.berkeley.edu,Elizabeth Abel's general research interest is...,NaN,"February 15, 2011","Jim Crow signs as symbols of subjugation, tro...",/news/jim-crow-signs-symbols-subjugation-troph...,"In the mid 1960s, landmark laws brought an o...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dor Abrahamson,/faculty/dor-abrahamson,"mathematical cognition,design-based research,m...",Graduate School of Education,Associate Professor of Cognition and Development,http://gse.berkeley.edu/people/dor-abrahamson,http://edrl.berkeley.edu/,dor@berkeley.edu,Dor Abrahamson studies the process of mathema...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Norman Abrahamson,/faculty/norman-abrahamson,"civil and environmental engineering,earthquake...",Department of Civil and Environmental Engineering,Adjunct Professor of Civil and Environmental E...,http://www.ce.berkeley.edu/faculty/faculty.php...,NaN,naa3@earthlink.net,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
grant_history_df = pd.read_csv('../scraped_data/research_grant_history.csv', delimiter=",")
grant_history_df.head()

/home/jerry/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Activity Type,Amount,Sponsor Class,Sponsor,Division,Department,Fund,UCB Award Number,PI Name,Project Begin Date,Project End Date,Title,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,Applied research,"$179,032",State of California,California Department of Health Care Services,School of Public Health,NaN,15952.0,021331-002,"Colford Jr, John M",7/1/2006,9/30/2006,DNS AIDS Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Basic research,"$154,578",State of California,California Department of Social Services,School of Social Welfare,NaN,15959.0,021362-002,"Needell, Barbara",7/1/2006,9/30/2006,Performance Indicators/California Children's S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Instruction,"$225,000",State of California,California Department of Social Services,School of Social Welfare,Social Welfare,15960.0,021363-002,"Midgley, James",7/1/2006,9/30/2006,Title IV-E Social Work Training Program,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Instruction,"$47,138",State of California,California Department of Social Services,School of Social Welfare,Social Welfare,15960.0,021363-002,"Midgley, James",7/1/2006,9/30/2006,Title IV-E Social Work Training Program,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Basic research,"$65,000",Federal,NIH National Institutes of Health - Miscellaneous,VC Res Other Research Units,The California Institute for Quantitative Bios...,78561.0,021425-002,"Keasling, Jay",7/1/2006,12/31/2006,Model-Driven Strain Engineering for Isoprenoid...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
nsf_grants_df = pd.read_csv('../scraped_data/nsf_grants.csv', delimiter="~")
nsf_grants_df.head()

,Headline,Due Date Start,Due Date End,Description,Link
0,Resource Implementations for Data Intensive Re...,None,2017-02-27,As part of NSF’s Cyberinfrastructure Framework...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...
1,Computer Science for All,None,2017-02-28,This program aims to provide all U.S.students ...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...
2,EPSCoR Research Infrastructure Improvement Tra...,None,2017-02-28,The Experimental Program to Stimulate Competit...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...
3,Industry-University Cooperative Research Cente...,None,2017-02-28,The Industry-University Cooperative Research C...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...
4,SOCIOLOGY PROGRAM - Doctoral Dissertation Rese...,None,2017-02-28,The Sociology Program supports basic research ...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5...


In [8]:
descriptions = grants_df['Description']

In [9]:
# Prepare grants.gov grants
cleaned = []
for desc in descriptions:
    try:
        desc = desc.lower()
        desc = desc.translate(str.maketrans('','',string.punctuation))
        desc = desc.translate(str.maketrans('','','1234567890'))
        first = filter(lambda x: x.lower() not in sw and len(x) > 3, desc.split())
        second = [wnl.lemmatize(x) for x in first]
        cleaned.append(second)
    except:
        cleaned.append(["Nothing"])
    

In [10]:
print(cleaned[0])

['office', 'press', 'public', 'diplomacy', 'bureau', 'south', 'central', 'asian', 'affair', 'department', 'state', 'coordination', 'bureau', 'european', 'eurasian', 'affair', 'bureau', 'education', 'cultural', 'affair', 'announces', 'open', 'competition', 'cooperative', 'agreement', 'implement', 'edmund', 'muskie', 'graduate', 'internship', 'program', 'herein', 'referred', 'muskie', 'internship', 'program', 'muskie', 'internship', 'program', 'designed', 'promote', 'mutual', 'understanding', 'foster', 'democratization', 'transition', 'market', 'economy', 'eurasia', 'intensive', 'professional', 'education', 'program', 'muskie', 'intern', 'begin', 'fourmonth', 'professional', 'internship', 'united', 'state', 'immediately', 'upon', 'completion', 'fulbright', 'foreign', 'student', 'program', 'internship', 'provide', 'opportunity', 'professional', 'development', 'well', 'reinforce', 'skill', 'gained', 'academic', 'study']


In [11]:
corpus = [' '.join(doc) for doc in cleaned]

In [12]:
grants_vectorizer = TfidfVectorizer(min_df=4)
grants_matrix = grants_vectorizer.fit_transform(corpus)

In [13]:
print(grants_matrix.shape)
print(len(grants_vectorizer.vocabulary_))

(41536, 19811)
19811


In [14]:
# Prepare NSF grants
cleaned = []
nsf_descriptions = nsf_grants_df['Description']
    
for desc in nsf_descriptions:
    try:
        desc = desc.lower()
        desc = desc.translate(str.maketrans('','',string.punctuation))
        desc = desc.translate(str.maketrans('','','1234567890'))
        first = filter(lambda x: x.lower() not in sw and len(x) > 3, desc.split())
        second = [wnl.lemmatize(x) for x in first]
        cleaned.append(second)
    except:
        cleaned.append(["Nothing"])
        
nsf_corpus = [' '.join(doc) for doc in cleaned]

In [15]:
nsf_vectorizer = TfidfVectorizer(min_df=4, vocabulary=grants_vectorizer.vocabulary_)
nsf_matrix = nsf_vectorizer.fit_transform(nsf_corpus)

In [16]:
nsf_matrix.shape

(484, 19811)

Creates faculty profiles. Currently takes all past research from research_grant_history.csv and vectorizes using vocabulary from grants_gov.csv and takes the mean (possibly not the best way to do this).

In [17]:
faculty_profiles = defaultdict(list)
for index, row in grant_history_df.iterrows():
    faculty_profiles[row["PI Name"]].append(row["Title"])

In [18]:
faculty_vectorizer = TfidfVectorizer(vocabulary=grants_vectorizer.vocabulary_)
for faculty in faculty_profiles:
    research = faculty_profiles[faculty]
    vectorized = faculty_vectorizer.fit_transform(research)
    faculty_profiles[faculty] = np.mean(vectorized, axis=0)

In [20]:
faculty_list = list(faculty_profiles.keys())

# Some sanity checks
print(faculty_profiles[faculty_list[1]].shape)
print(grants_matrix[0][0].shape)
print(len(faculty_profiles))

(1, 19811)
(1, 19811)
2109


In [77]:
top_matches = {}
test_faculty = random.sample(faculty_list, 10)
for faculty in test_faculty:
    profile = faculty_profiles[faculty]
    matches = []
    for j in range(grants_matrix.shape[0]):
        grant = grants_matrix[j]
        matches.append((np.dot(grant.toarray(), profile.T), j))
    top_matches[faculty] = [x[1] for x in sorted(matches)[-5:]]


In [152]:
print(top_matches)

{'Sundkvist, David J': [9957, 37943, 20002, 32981, 21936], 'Bentley, George': [38718, 5696, 15109, 1614, 8264], 'Taymor, Kenneth': [17562, 28513, 5386, 7079, 27997], 'Frenklach, Michael Y': [19576, 15128, 37350, 3229, 10406], 'Williams, Caroline Margaret': [27118, 10406, 36750, 2607, 28468], 'Ferme, Mariane C': [24590, 2535, 20843, 20739, 29210], 'Agol, Ian': [27401, 36870, 36872, 30466, 27403], 'Treib, Edward Marc': [24554, 30555, 32135, 27364, 40287], 'Fulton, Brent D': [7250, 31865, 23263, 7857, 4674], 'Price Jr, P Buford': [16761, 5953, 29580, 17977, 29883]}


In [153]:
matches = defaultdict(list)
for faculty in top_matches:
    for index in top_matches[faculty]:
        matches[faculty].append(grants_df[index:index+1])

In [154]:
# Grant matches for first faculty in top_matches for grants_gov
faculty = list(matches.keys())[0]
print(faculty)
for match in matches[faculty]:
    display(match)

Sundkvist, David J


,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
9957,281050,State Energy Program 2016 Competitive Awards,DE-FOA-0001415,D,CA,EN,NaN,81.119,00,NaN,DOE-GFO,Golden Field Office,1192016,3312016.0,1192016,2.0,1.0,0.0,0.0,The Office of Energy Efficiency and Renewable Energy&#8217;s (EERE&#8217;s) State Energy Program...,Synopsis 1,Yes,7062016.0,rochelle.hass@ee.doe.gov,rochelle.hass@ee.doe.gov,Rochelle M. Hass 720-356-1559&lt;br/&gt;rochelle.hass@ee.doe.gov&lt;br/&gt;,https://eere-exchange.energy.gov,https://eere-exchange.energy.gov,NaN,NaN


,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
37943,254536,State Energy Program 2014 Competitive Awards,DE-FOA-0001073,D,CA,EN,NaN,81.119,00,NaN,DOE-GFO,Golden Field Office,4302014,6302014.0,4302014,0.0,0.0,5000000.0,NaN,The Office of Energy Efficiency and Renewable Energy?s (EERE?s) State Energy Program (SEP) seeks...,Synopsis 2,Yes,9302014.0,deborah.reynolds@go.doe.gov,deborah.reynolds@go.doe.gov,Deborah J. Reynolds&lt;br/&gt;deborah.reynolds@go.doe.gov&lt;br/&gt;,https://www.fedconnect.net,https://www.fedconnect.net,NaN,NaN


,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
20002,101073,State Energy Program (SEP) PY 2011 Formula Award Funding Opportunity Announcement,DE-FOA-0000507,D,G,EN,NaN,81.041,00,NaN,DOE-NETL,National Energy Technology Laboratory,6232011,8192011.0,8052011,0.0,0.0,39000000.0,56.0,DOE&apos;s State Energy Program (SEP) is seeking to provide grants to state energy offices in al...,Synopsis 4,Yes,NaN,sheldon.funk@netl.doe.gov,sheldon.funk@netl.doe.gov,"Sheldon E. Funk, 304-285-0204 &lt;br/&gt;sheldon.funk@netl.doe.gov &lt;br/&gt;",https://www.fedconnect.net/FedConnect/,FedConnect,NaN,NaN


,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
32981,53209,State Energy Program (SEP) PY 2010 Formula Award Funding Opportunity Announcement,DE-FOA-0000308,D,G,EN,NaN,81.041,00,NaN,DOE-NETL,National Energy Technology Laboratory,3262010,8312010.0,3262010,NaN,NaN,25000000.0,56.0,DOE&apos;s State Energy Program (SEP) is seeking to provide grants to state energy offices in al...,Synopsis 1,Yes,NaN,sheldon.funk@netl.doe.gov,sheldon.funk@netl.doe.gov,"Sheldon E. Funk, Contract Specialist, 304-285-0204&lt;br/&gt;sheldon.funk@netl.doe.gov&lt;br/&gt;",https://www.fedconnect.net,FedConnect,NaN,NaN


,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,AgencyCode,AgencyName,PostDate,CloseDate,LastUpdatedDate,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
21936,105833,State Energy Program 2011 Competitive Awards,DE-FOA-0000533,D,"CA,G",EN,NaN,81.119,00,NaN,DOE-GFO,Golden Field Office,7152011,8112011.0,7152011,0.0,0.0,0.0,NaN,The Department of Energy (DOE) State Energy Program (SEP) is seeking to provide grants to state ...,Synopsis 1,Yes,NaN,yolanda.ramirez@go.doe.gov,yolanda.ramirez@go.doe.gov,Yolanda C. Ramirez&lt;br/&gt;yolanda.ramirez@go.doe.gov&lt;br/&gt;,https://www.fedconnect.net/,https://www.fedconnect.net/https://www.fedconnect.net/,NaN,NaN


In [25]:
# Match percentages
# TODO

In [156]:
nsf_top_matches = {}
nsf_test_faculty = random.sample(faculty_list, 10)
for faculty in nsf_test_faculty:
    profile = faculty_profiles[faculty]
    matches = []
    for j in range(nsf_matrix.shape[0]):
        grant = nsf_matrix[j]
        matches.append((np.dot(grant.toarray(), profile.T), j))
    nsf_top_matches[faculty] = [x[1] for x in sorted(matches)[-5:]]
    
nsf_matches = defaultdict(list)
for faculty in nsf_top_matches:
    for index in nsf_top_matches[faculty]:
        nsf_matches[faculty].append(nsf_grants_df[index:index+1])

In [157]:
# Grant matches for first faculty in nsf_top_matches for NSF grants
faculty_nsf = list(nsf_matches.keys())[0]
print(faculty)
for match in nsf_matches[faculty_nsf]:
    display(match)

Tremsin, Anton


,Headline,Due Date Start,Due Date End,Description,Link
125,Cooperative Studies Of The Earth's Deep Interior,None,2017-09-25,"The Division of Earth Sciences (EAR) invites the submission of proposals for collaborative, inte...",https://nsf.gov/funding/pgm_summ.jsp?pims_id=11702&org=NSF&sel_org=NSF&from=fund


,Headline,Due Date Start,Due Date End,Description,Link
421,"Science, Technology, Engineering, and Mathematics Advanced Global Institutes Program",Not listed,Not listed,The Pan-American Advanced Studies Institutes (PASI) Program is a jointly supported initiative be...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=5327&org=NSF&sel_org=NSF&from=fund


,Headline,Due Date Start,Due Date End,Description,Link
422,Panel Study of Income Dynamics (PSID) Competition,Not listed,Not listed,A Competition of the Panel Study of Income Dynamics (PSID),https://nsf.gov/funding/pgm_summ.jsp?pims_id=13460&org=NSF&sel_org=NSF&from=fund


,Headline,Due Date Start,Due Date End,Description,Link
63,Law & Social Sciences,None,2017-08-01,The Law & Social Sciences Program considers proposals that address social scientific studies of ...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=504727&org=NSF&sel_org=NSF&from=fund


,Headline,Due Date Start,Due Date End,Description,Link
251,Law & Social Sciences,None,2017-08-01,The Law & Social Sciences Program considers proposals that address social scientific studies of ...,https://nsf.gov/funding/pgm_summ.jsp?pims_id=504727&org=NSF&sel_org=NSF&from=fund


In [76]:
# Links to the matches
pd.options.display.max_colwidth = 100
for match in nsf_matches[faculty]:
    print(match['Link'].to_string())

356    https://nsf.gov/funding/pgm_summ.jsp?pims_id=503172&org=NSF&sel_org=NSF&from=fund
82    https://nsf.gov/funding/pgm_summ.jsp?pims_id=12729&org=NSF&sel_org=NSF&from=fund
290    https://nsf.gov/funding/pgm_summ.jsp?pims_id=12729&org=NSF&sel_org=NSF&from=fund
120    https://nsf.gov/funding/pgm_summ.jsp?pims_id=13358&org=NSF&sel_org=NSF&from=fund
325    https://nsf.gov/funding/pgm_summ.jsp?pims_id=12728&org=NSF&sel_org=NSF&from=fund


<h1> Clustering approach </h1>

Use random set of training grants from grants.gov grants and assigns them to professors using one of the two approaches below.
Then takes a test sample from NSF grants and see which professors are matched with it.

<b> Only run one of the two cells below </b>

In [169]:
# This grant assignment approach assigns professors the grants that they have the highest similiarity with.
# Multiple professors may be assigned to a single grant

training_size = 1000
training_grants = sklearn.utils.shuffle(grants_matrix[:])[:training_size]
progress = 0
faculty_to_grants = {}
for faculty in faculty_profiles:
    profile = faculty_profiles[faculty]
    matches = []
    for j in range(training_grants.shape[0]):
        grant = training_grants[j]
        matches.append((np.dot(grant.toarray(), profile.T), j))
    faculty_to_grants[faculty] = [x[1] for x in sorted(matches)[-5:]]
    if progress % 250 == 0:
        print(str(progress) + "/" + str(len(faculty_profiles)))
    progress += 1
print(str(len(faculty_profiles)) + "/" + str(len(faculty_profiles)))

0/2109


KeyboardInterrupt: 

In [174]:
# This grant assignment approach assigns professors the grants that they have the highest similiarity with.
# One-to-one matching with grants and professors
# If using this matching approach, a faculty may appear muliple times when testing matches. Maybe this can be used
# as a signal that this faculty is an exceptionally good match?

# Issue: It seems like certain faculty dominate in terms of grants assigned.

training_size = 1000
training_grants = sklearn.utils.shuffle(grants_matrix[:])[:training_size]
progress = 0
faculty_to_grants = defaultdict(list)
for i in range(training_grants.shape[0]):
    grant = training_grants[i]
    top_faculty = None
    top_score = 0
    for faculty in faculty_profiles:
        profile = faculty_profiles[faculty]
        if np.dot(grant.toarray(), profile.T) > top_score:
            top_faculty = faculty
    faculty_to_grants[top_faculty].append(i)
    if progress % 250 == 0:
        print(str(progress) + "/" + str(training_size))
    progress += 1
print(str(training_size) + "/" + str(training_size))

0/1000
250/1000
500/1000
750/1000
1000/1000


Takes a random NSF grant and matches it with faculty.
Prints the test NSF grant information, link to the grant, and the faculty that it matched with.

In [188]:
# Take a random NSF grant and match with professors
random_index = random.sample((range(nsf_matrix.shape[0])), 1)[0]
test_grant = nsf_matrix[random_index]
closest_grants = []
for index in range(training_grants.shape[0]):
    training_grant = training_grants[index]
    # Uses dot similarity
    closest_grants.append((np.dot(training_grant, test_grant.T).toarray()[0], index))
    
# Finds the 10 grants that are closest to the test grant
closest_grants = [x[1] for x in sorted(closest_grants)[-10:]]

clustering_matches = []
for faculty in faculty_to_grants:
    faculty_grants = faculty_to_grants[faculty]
    for closest_grant in closest_grants:
        if closest_grant in faculty_grants:
            clustering_matches.append(faculty)

# Print out test grant, link to grant, and matched faculty
test_grant_full = nsf_grants_df[random_index:random_index+1]
display(test_grant_full)
for faculty in clustering_matches:
    print(faculty)
    
print(test_grant_full['Link'].to_string())

,Headline,Due Date Start,Due Date End,Description,Link
31,Advanced Biomanufacturing of Therapeutic Cells,None,2017-04-17,"In recent years, somatic cells as therapeutic agents have provided new treatment approaches for ...",https://nsf.gov/funding/pgm_summ.jsp?pims_id=505376&org=NSF&sel_org=NSF&from=fund


Flanagan, John
Flanagan, John
Flanagan, John
Yao, Jie
Dawson, Todd
Dawson, Todd
Dawson, Todd
Dawson, Todd
Roychowdhury, Jaijeet
Agapitov, Oleksiy
31    https://nsf.gov/funding/pgm_summ.jsp?pims_id=505376&org=NSF&sel_org=NSF&from=fund
